<a href="https://colab.research.google.com/github/DKojen/Matrix/blob/master/matrix_one/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 3.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook


In [5]:
cd "/content/drive/My Drive/Colab Notebooks/Matrix"

/content/drive/My Drive/Colab Notebooks/Matrix


In [8]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats):
  x = df[feats].values
  y = df['prices_amountmin']

  model = DecisionTreeRegressor(max_depth=5)

  scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [18]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

-58 to średnie odchylenie bezwzględne. Model średnio zgaduje z dokładnością +/- 58USD. Miara pozwala nam zobaczyć jak model dopasowwywuje się do danych orginalnych. Możemy wziąć inną cechę żeby porównać jej wynik średniego odchylenia bezwzględnego np jeśli dla features będzie to +/-30USD to model z tą cechą będzie lepszy. Możemy również użyc zupełnie innego modelu.

In [16]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state=0)
run_model(['brand_cat'], model)

TypeError: ignored

In [20]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [22]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [32]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [34]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))
len(keys)

476

In [35]:
def get_name_feat(key):
  return 'feat_' + key
for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [36]:
df.columns

Index(['Unnamed: 0', 'id', 'asins', 'brand', 'categories', 'colors', 'count',
       'dateadded', 'dateupdated', 'descriptions',
       ...
       'feat_productgroup', 'feat_shipping weight',
       'feat_elastic banded waist', 'feat_is waterproof', 'feat_shade',
       'feat_has expiration', 'feat_country of origin components:',
       'feat_closure style', 'feat_upper material', 'feat_lens color'],
      dtype='object', length=527)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key]= df[False == df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100


In [45]:
{k:v for k,v in keys_stat.items() if v> 30} #szukamy atrybutów które występują najczęściej wśród danych

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]
df['feat_shape_cat'] = df['feat_shape'].factorize()[0]


In [48]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df [df.brand == df.feat_brand ].shape

(8846, 532)

In [72]:
feats_cat = [x for x in df.columns if 'cat' in x ]
feats_cat

['categories',
 'brand_cat',
 'feat_clothing category',
 'feat_recommended location',
 'feat_certifications and listings',
 'feat_shoe category',
 'feat_location - city/state',
 'feat_location - country',
 'feat_multi pack indicator',
 'feat_fabrication',
 'feat_catalog',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat']

In [0]:
feats = ['brand_cat','feat_shape_cat', 'feat_style_cat', 'feat_brand_cat','feat_gender_cat','feat_material_cat']
#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats,model)

In [81]:
x = df[feats].values
y = df['prices_amountmin'].values
m = RandomForestRegressor (max_depth=5, n_estimators=100, random_state=0)
m.fit(x,y)

print(result)
perm = PermutationImportance(m,random_state=1).fit(x,y);
eli5.show_weights(perm, feature_names=feats)

(-58.354960581809095, 4.26089691178124)


Weight,Feature
0.2618 ± 0.0112,brand_cat
0.1092 ± 0.0115,feat_material_cat
0.0230 ± 0.0031,feat_gender_cat
0.0217 ± 0.0017,feat_brand_cat
0.0137 ± 0.0012,feat_shape_cat
0.0042 ± 0.0010,feat_style_cat


Okazuje się że feat_manufacturer part number nie jest istotną cechą zatem ją usuwamy z modelu

In [59]:
df['brand'].value_counts(normalize=True)

nike             0.097210
puma             0.033315
ralph lauren     0.028775
vans             0.021116
new balance      0.020295
                   ...   
sessions         0.000055
neumann          0.000055
walk over        0.000055
plixio           0.000055
caribbean joe    0.000055
Name: brand, Length: 1732, dtype: float64

In [60]:
df[ df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [64]:
df[ df['brand'] == 'nike'].features_parsed.sample().values

array([{'sport': 'skate shoes', 'occasion': 'skate shoes', 'material': 'textile', 'gender': 'men', 'shoe size': '12 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'textile', 'color': 'black/dark grey gm light brown', 'model': '729825 004', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'skateboarding shoes', 'manufacturer part number': '729825 004', 'brand': 'nike', 'shoe width': 'd(m)'}],
      dtype=object)